This is a demo notebook of some of the features of IEtools.py.

IEtools includes tools to read FRED economic data

https://fred.stlouisfed.org/

in either csv or xls formats. IEtools also includes tools for
fitting information equilibrium parameters and constructing
dynamic equilibrium models (see Dynamic Equilibrium Examples.ipynb).

The basic information equilibrium condition between two variables
A and B is given by

p = dA/dB = k A/B

with information transfer (IT) index k. In 'general equilibrium' 
(where A or B is not changing faster than the other), we have the 
solution

A = a B^k

or

log(A) = k log(B) + c

IEtools has a function to solve for these parameters. In the
information equilibrium condition, p = dA/dB is the abstract price.
In 'general equilibrium', we have

p = k a B^(k-1)

The continuously compounded growth rates of these variables are 
also related. If the growth rate of A is g_a, B is g_b, 
and p is g_p then:

g_a = k g_b

g_p = (k-1) g_b

g_p = g_a - g_b

This notebook shows some of these results for GDP and labor supply, 
showing Okun's law along the way.

IEtools was tested using Python 3.6 as part of the Anaconda 4.4.0 
package. All dependencies are included in Anaconda 4.4.0.

In [ ]:
import numpy as np
import IEtools
import pylab as pl
%pylab inline

# Read in the files

In [ ]:
filename1='C:/econdata/GDP.xls'
filename2='C:/econdata/PAYEMS.xls'
filename3='C:/econdata/CPIAUCSL.xls'

In [ ]:
gdp = IEtools.FREDxlsRead(filename1)
lab = IEtools.FREDxlsRead(filename2)
cpi = IEtools.FREDxlsRead(filename3)

Here's a plot of nominal GDP

In [ ]:
pl.plot(gdp['interp'].x,gdp['interp'](gdp['interp'].x))
pl.ylabel(gdp['name']+' [G$]')
pl.yscale('log')
pl.show()

And here is nominal GDP growth

In [ ]:
pl.plot(gdp['growth'].x,gdp['growth'](gdp['growth'].x))
pl.ylabel(gdp['name']+' growth [%]')
pl.show()

# Fit information equilibrium parameters

Here we take the information equilibrium model with A = nominal GDP, B = labor employed (PAYEMS), and p = CPI (all items). First we solve for the IT index and show the model. Note: this is a simple model with limited accuracy.

In [ ]:
result = IEtools.fitGeneralInfoEq(gdp['data'],lab['data'], guess=[1.0,0.0])
print(result)
print('IT index = ',np.round(result.x[0],decimals=2))
time=gdp['interp'].x
pl.plot(time,np.exp(result.x[0]*np.log(lab['interp'](time))+result.x[1]),label='model')
pl.plot(time,gdp['interp'](time),label='data')
pl.yscale('log')
pl.ylabel(gdp['name']+' [G$]')
pl.legend()
pl.show()

And we can show the relationship between the growth rates (i.e. compute the inflation rate equal to the growth rate of the CPI)

In [ ]:
time=gdp['data'][:,0]

der1=gdp['growth'](time)-lab['growth'](time)
der2=cpi['growth'](time)
pl.plot(time,der1,label='model')
pl.plot(time,der2,label='data')
pl.legend()
pl.show()

Additionally, rearranging the terms and looking at the growth rate, we can show a form of Okun's law. Since g_p = g_a - g_b, we can say g_b = g_a - g_p. The right hand side of the last equation when A is nominal GDP and p is the CPI is the CPI-deflated real GDP growth. Okun's law is an inverse relationship between the change in unemployment and RGDP growth, but in our case we will look at the direct relationship of RGDP growth and change in employment (PAYEMS).

In [ ]:
time=gdp['data'][:,0]

der1=gdp['growth'](time)-cpi['growth'](time)
der2=lab['growth'](time)
pl.plot(time,der1,label='model')
pl.plot(time,der2,label='data')
pl.legend()
pl.show()